# Idea
The idea is that we predict rotated linear dimension position from lines and text from the drawing.

# Thoughts
Basic logic is we split dataset to input lines or texts positions StartPoint, EndPoint, Position XYZ and predict dimension extension line poistion XLine1Point, XLine2Point XYZ.

We going to group samples by FileId. That is each sample will contain variable length data (attributes of variable number of  lines and text) and variable output data (variable number of dimensions).

I intend to [use RNN for it](https://medium.com/dair-ai/building-rnns-is-fun-with-pytorch-and-google-colab-3903ea9a3a79)

In [401]:
# https://stackoverflow.com/questions/16249736/how-to-import-data-from-mongodb-to-pandas
import pymongo
import pandas as pd
import numpy as np
from pymongo import MongoClient
from processing import *

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import time

In [402]:
test_data = pd.read_pickle('test_dataset.pickle')
# BUG: Dataset is filtered, we need all data from SOME files, not SOME data from All Files
test_data

,ClassName,FileId,XLine1Point.X,XLine1Point.Y,XLine1Point.Z,XLine2Point.X,XLine2Point.Y,XLine2Point.Z,StartPoint.X,StartPoint.Y,StartPoint.Z,EndPoint.X,EndPoint.Y,EndPoint.Z,Position.X,Position.Y,Position.Z
0,AcDbText,006cb2dd-6f18-4203-8b70-fc865c08105a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.695776,35.642180,0.0
1,AcDbText,006cb2dd-6f18-4203-8b70-fc865c08105a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.733352,10.433374,0.0
2,AcDbText,006cb2dd-6f18-4203-8b70-fc865c08105a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.733352,6.266707,0.0
3,AcDbText,006cb2dd-6f18-4203-8b70-fc865c08105a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.733352,22.584136,0.0
4,AcDbText,006cb2dd-6f18-4203-8b70-fc865c08105a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.733352,18.417470,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89231,AcDbText,09bb1674-bff4-4ac7-a86d-60a9600ce899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.169657,92.228915,0.0
89232,AcDbLine,09bb1674-bff4-4ac7-a86d-60a9600ce899,NaN,NaN,NaN,NaN,NaN,NaN,20.2884,155.228915,0.0,0.2884,155.228915,0.0,NaN,NaN,NaN
89233,AcDbLine,09bb1674-bff4-4ac7-a86d-60a9600ce899,NaN,NaN,NaN,NaN,NaN,NaN,5.2884,145.228915,0.0,20.2884,145.228915,0.0,NaN,NaN,NaN
89234,AcDbText,09bb1674-bff4-4ac7-a86d-60a9600ce899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,316.663400,1.478915,0.0


# Dataset implementation

In [403]:
# https://blog.floydhub.com/a-beginners-guide-on-recurrent-neural-networks-with-pytorch/
import torch
from torch import nn
from torch.utils.data import Dataset, SubsetRandomSampler
from torch.nn import functional as F

device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
    
device

device(type='cpu')

In [404]:
x_columns = [
    #'ClassName', 
    'StartPoint.X', 'StartPoint.Y', 'StartPoint.Z',
    'EndPoint.X', 'EndPoint.Y', 'EndPoint.Z',
    'Position.X', 'Position.Y', 'Position.Z']

y_columns = [
    #'ClassName', 
    'XLine1Point.X', 'XLine1Point.Y','XLine1Point.Z', 
    'XLine2Point.X', 'XLine2Point.Y', 'XLine2Point.Z']

ent_features = len(x_columns)
dim_features = len(y_columns)

In [405]:
class EntityDataset(Dataset):
    def __init__(self, pandasData):
        self.data = pandasData
        self.groupped = pandasData.groupby('FileId')
        self.keys = list(self.groupped.groups.keys() )
        
    def __len__(self):
        return len(self.keys)
    
    def __getitem__(self, index):
        # https://stackoverflow.com/questions/45147100/pandas-drop-columns-with-all-nans
        
        x = self.groupped.get_group(self.keys[index])[x_columns]
        x = x.dropna( how ='all')
        y = self.groupped.get_group(self.keys[index])[y_columns]
        y = y.dropna(how = 'all')
        #print(self.keys[index])
        # if not y.empty:
        #    print(x, y)
        
        return torch.tensor(np.array(x.values)), torch.tensor(np.array(y.values))

In [406]:
entities = EntityDataset(test_data)
for c,d in entities:
    # print(c.shape, d.shape)
    assert( c.shape[0] != 0)
    

In [407]:
batch_size = 2

data_len = len(entities)

validation_fraction = 0.2
test_fraction       = 0.3

val_split  = int(np.floor(validation_fraction * data_len))
test_split = int(np.floor(test_fraction * data_len))
indices = list(range(data_len))
np.random.seed(228)

np.random.shuffle(indices)

val_indices   = indices[:val_split]
test_indices  = indices[val_split:test_split]
train_indices = indices[test_split:]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler   = SubsetRandomSampler(val_indices)
test_sampler  = SubsetRandomSampler(test_indices)

# https://stackoverflow.com/questions/64586575/adding-class-objects-to-pytorch-dataloader-batch-must-contain-tensors
def custom_collate(sample):
    x = []
    y = []

    for (xx, yy) in sample:
        #if torch.count_nonzero(xx) > 0:
            # print(xx)
        x.append(xx)
        #if torch.count_nonzero(yy) > 0:
        y.append(yy)
    #print(len(x), len(y))
    return x, y

train_loader = torch.utils.data.DataLoader(entities, batch_size = batch_size, sampler = train_sampler, collate_fn=custom_collate)
val_loader   = torch.utils.data.DataLoader(entities, batch_size = batch_size, sampler = val_sampler, collate_fn=custom_collate)
test_loader  = torch.utils.data.DataLoader(entities, batch_size = batch_size, sampler = test_sampler, collate_fn=custom_collate)

In [408]:
(a,b) = next(iter(train_loader))
print(len(a),len(b))
for (x,y) in iter(train_loader):
    for xx in x:
        print(xx.shape)
        pass
    for yy in y:
        print(yy.shape)
    # print(a.shape,b.shape)
    break

2 2
torch.Size([990, 9])
torch.Size([922, 9])
torch.Size([1, 6])
torch.Size([3, 6])


# Model and training


In [409]:
class RnnEncoder(nn.Module):
    def __init__(self, ent_features, learned_size, n_neurons = 16):
        super(RnnEncoder, self).__init__()

        self.rnn = nn.RNN(input_size = ent_features, hidden_size = learned_size)
        self.learned_size = learned_size
        self.ent_features = ent_features
        
        self.fcn = nn.Sequential(
            nn.Linear(learned_size, 1),
            nn.ReLU()
        )
        
    def init_hidden(self,batch_size):
        self.hidden = torch.zeros(1, batch_size, self.learned_size)
        
    def forward(self, x):
        # input of shape (seq_len, batch, input_size)
        batch_size = len(x)
        
        hiddens = torch.zeros([batch_size, self.learned_size], dtype = torch.float32, device = device)
        outs = torch.zeros([batch_size, 1], dtype = torch.int32, device = device)
        for j in range(batch_size):
            f = x[j]
            entities_count = f.shape[0]
            self.init_hidden(entities_count)
            inp = torch.zeros([entities_count, 1, self.ent_features], dtype = torch.float32, device = device)
            # print(inp.shape, f.shape)
            inp[:,0,:] = f[:,:]
            
            outp, h_n = self.rnn(inp)
            # print('h_N', h_n.shape)
            # print('cell out', outp.shape)
            n = self.fcn(outp)
            
            # https://stackoverflow.com/questions/48005152/extract-the-count-of-positive-and-negative-values-from-an-array
            cou = np.sum(np.array(n.detach().numpy()) > 0, axis = 0)
            # print(cou)
            outs[j] = torch.tensor(cou) 
            hiddens[j] = h_n
            j = j + 1
                
        return outs, hiddens
            
class RnnDecoder(nn.Module):
    def __init__(self, learned_size, dim_features):
        super(RnnDecoder, self).__init__()
        self.rnn = nn.RNN(input_size = learned_size, hidden_size = dim_features)
        self.learned_size = learned_size
        self.dim_features = dim_features
        
    def forward(self, x, out_counts):
        '''Receives tensor with hidden state and number of dimensions tensor per each file.
            Returns a list of tensors with dimension parameters'''
        # input of shape (seq_len, batch, input_size)
        
        result = []
        for j in range(batch_size):
            dim_count = out_counts[j]
            outs = torch.zeros([dim_count,dim_features])
            if dim_count > 0:
                inp = torch.zeros([dim_count, 1, self.learned_size], dtype=torch.float32, device = device)
                xj = x[j]
                # print(inp.shape, xj.shape)
                inp[:,0] = xj
                outs, hiddens = self.rnn(inp)
                print("decoder outs.shape", outs.shape)
                # we remove batch dimension, as batches here are list
                # https://pytorch.org/docs/stable/generated/torch.squeeze.html
                outs = outs.squeeze(dim = 1)
            result.append(outs)
        
        return result

In [410]:
rnn_encoder = RnnEncoder(ent_features, 1024).to(device)
rnn_decoder = RnnDecoder(1024, dim_features).to(device)
(x,y) = next(iter(train_loader))
print('inputs','x:',len(x),'x',x[0].shape,'y:',len(y),'x',y[0].shape)
outs_numbers, learned = rnn_encoder(x)
print('learned shape:', learned.shape)
print("outs shape:", outs.shape, 'outs_value', outs)
# make sure something is passed
outs[0] = 42
decoded = rnn_decoder(learned, outs)
print('decoded:',len(decoded),'x', decoded[0].shape)

inputs x: 2 x torch.Size([29404, 9]) y: 2 x torch.Size([367, 6])


/home/mk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in greater


learned shape: torch.Size([2, 1024])
outs shape: torch.Size([2, 1]) outs_value tensor([[42],
        [ 0]], dtype=torch.int32)
decoder outs.shape torch.Size([42, 1, 6])
decoded: 2 x torch.Size([42, 6])


In [424]:
# test run all data samples
for (x,y) in iter(train_loader):
    try:
        outs_numbers, learned = rnn_encoder(x)
        results = rnn_decoder(outs_numbers, learned)
    except Exception as e:
        # print(e)
        for xx in x:
            print(xx.shape)
        print(outs_numbers.shape)
        raise(e)
        break

6


/home/mk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in greater


torch.Size([603, 9])
torch.Size([955, 9])
torch.Size([2, 1])


TypeError: zeros(): argument 'size' must be tuple of ints, but found element of type Tensor at pos 1

In [412]:
import numpy.linalg as LA
# https://stackoverflow.com/questions/47060685/chamfer-distance-between-two-point-clouds-in-tensorflow
def array2samples_distance(array1, array2):
    """
    arguments: 
        array1: the array, size: (num_point, num_feature)
        array2: the samples, size: (num_point, num_feature)
    returns:
        distances: each entry is the distance from a sample to array1 
    """
    num_point, num_features = array1.shape
    expanded_array1 = np.tile(array1, (num_point, 1))
    expanded_array2 = np.reshape(
            np.tile(np.expand_dims(array2, 1), 
                    (1, num_point, 1)),
            (-1, num_features))
    distances = LA.norm(expanded_array1-expanded_array2, axis=1)
    distances = np.reshape(distances, (num_point, num_point))
    distances = np.min(distances, axis=1)
    distances = np.mean(distances)
    return distances

def chamfer_distance_numpy(array1, array2):
    batch_size, num_point, num_features = array1.shape
    dist = 0
    for i in range(batch_size):
        av_dist1 = array2samples_distance(array1[i], array2[i])
        av_dist2 = array2samples_distance(array2[i], array1[i])
        dist = dist + (av_dist1+av_dist2)/batch_size
    return dist

from sklearn.neighbors import KDTree

def chamfer_distance_sklearn(array1,array2):
    batch_size, num_point = array1.shape[:2]
    dist = 0
    for i in range(batch_size):
        tree1 = KDTree(array1[i], leaf_size=num_point+1)
        tree2 = KDTree(array2[i], leaf_size=num_point+1)
        distances1, _ = tree1.query(array2[i])
        distances2, _ = tree2.query(array1[i])
        av_dist1 = np.mean(distances1)
        av_dist2 = np.mean(distances2)
        dist = dist + (av_dist1+av_dist2)/batch_size
    return dist

In [413]:
import math

def my_chamfer_distance(out, y):
    loss = torch.zeros(1, dtype = torch.float32, requires_grad = True)

    for i in range(len(y)):
        # we need extra dimension for batch
        xxx = torch.unsqueeze(out[i], dim = 0)
        yyy = torch.unsqueeze(y[i],dim = 0)    
        # print(yyy.shape)
        # print(xxx.shape)    

        if xxx.shape[1] == 0 and yyy.shape[1] == 0:
            # loss += 0
            continue
        if xxx.shape[1] == 0:
            loss = loss + math.inf
            continue
        if yyy.shape[1] == 0:
            loss = loss + math.inf
            continue
        # https://discuss.pytorch.org/t/leaf-variable-was-used-in-an-inplace-operation/308
        curr_loss = chamfer_distance_sklearn(xxx.detach().numpy(), yyy.detach().numpy()).sum()
        loss = loss + curr_loss
    return loss

# print(my_chamfer_distance(decoded, y))
a = [torch.randn([3,6]), torch.randn([2,6])]
b = [torch.randn([4,6]), torch.randn([1,6])]
# print(a,b)
print(my_chamfer_distance(a,b))

class ChamferDistance(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, input, target):
        return my_chamfer_distance(input, target)

tensor([8.7649], grad_fn=<AddBackward0>)


In [414]:
def plot_history(loss_history, train_history, val_history):
    plt.ylabel('Accuracy @ epoch')
    
    train, = plt.plot(train_history)
    train.set_label("train")
    validation, = plt.plot(val_history)
    validation.set_label("validation")

    plt.legend()
    plt.show()

In [415]:
def train_model(encoder, decoder, train_loader, val_loader, loss, decoder_opt,encoder_opt, epochs):
    start = time.time()
    
    loss_history = []
    train_istory = []
    val_history  = []
    train_accuracy = 0.0
    val_accuracy = 0.0
    
    for epoch in range(epochs):
        encoder.train()
        decoder.train()
        
        loss_accumulated = 0
        
        i_step = 1
        for _, (x,y) in enumerate(train_loader):
            
            decoder_opt.zero_grad()
            encoder_opt.zero_grad()
            
            outs_num, learned = encoder(x)
            
            decoded = decoder(learned, outs_num)
            loss_value = loss(decoded, y)
            
            loss_value.backward()
            decoder_opt.step()
            encoder_opt.step()
            
            loss_accumulated += loss_value
            
            #val_accuracy = compute_accuracy(model, val_loader)
            print('[{0} @ {4:.1f} sec] Loss: {1:4f} Train err: {2:2.2f}% Val err: {3:2.2f}%'. format(
                epoch,
                loss_value.item(),
                (1 - train_accuracy) * 100,
                (1 - val_accuracy) * 100,
                time.time() - start
            ))
        
        # loss_history.append(float(loss_accum / i_step))
        # val_history.append(val_accuracy)
        # clear_output(wait=True)
        
        i_step += 1
        
    plot_history(loss_history, train_history, val_history)

In [416]:
lr = 1e-3
epochs = 3
decoder_optimizer = torch.optim.Adam(rnn_decoder.parameters(), lr = lr)
encoder_optimizer = torch.optim.Adam(rnn_encoder.parameters(), lr=lr)
loss = ChamferDistance()

train_model(
    encoder = rnn_encoder, 
    decoder = rnn_decoder, 
    train_loader = train_loader,
    val_loader = val_loader,
    loss = loss,
    decoder_opt = decoder_optimizer,
    encoder_opt = encoder_optimizer,
    epochs = epochs)

/home/mk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in greater


[0 @ 0.8 sec] Loss:  inf Train err: 100.00% Val err: 100.00%
[0 @ 3.7 sec] Loss:  inf Train err: 100.00% Val err: 100.00%
[0 @ 4.3 sec] Loss:  inf Train err: 100.00% Val err: 100.00%
[0 @ 5.0 sec] Loss:  inf Train err: 100.00% Val err: 100.00%
[0 @ 5.8 sec] Loss:  inf Train err: 100.00% Val err: 100.00%
[0 @ 16.7 sec] Loss:  inf Train err: 100.00% Val err: 100.00%
[0 @ 19.2 sec] Loss:  inf Train err: 100.00% Val err: 100.00%
[0 @ 20.5 sec] Loss:  inf Train err: 100.00% Val err: 100.00%
[0 @ 21.1 sec] Loss:  inf Train err: 100.00% Val err: 100.00%
[0 @ 22.7 sec] Loss:  inf Train err: 100.00% Val err: 100.00%


IndexError: index 1 is out of bounds for dimension 0 with size 1

In [ ]:
torch.cuda.empty_cache()

model.eval()
for (x,y) in test_loader:
    with torch.no_grad():
        output = model.forward(x.to(device))
        accuracy = compute_accuracy(model, test_loader)